In [ ]:
#normal diamond structure
#%%writefile PIT.py
import math
import meep as mp
from meep import mpb
#!rm *.h5
num_bands = 1

#k_points = [mp.Vector3(),
#            mp.Vector3(0.0  ,0.5  ,0.5  ),  #X
#            mp.Vector3(0.0  ,0.625,0.375),  #U
#            mp.Vector3(0.0  ,0.5  ,0.0  ),  #L
#            mp.Vector3(0.0  ,0.0  ,0.0  ),  #Gamma
#            mp.Vector3(0.0  ,0.5  ,0.5  ),  #X
#            mp.Vector3(0.25 ,0.75 ,0.5  ),  #W
#            mp.Vector3(0.375,0.75 ,0.375),  #K
#            mp.Vector3()]          

k_points = [mp.Vector3(),
            mp.Vector3(0.0  ,0.5  ,0.5  ),  #X
            mp.Vector3(0.5  ,0.625,0.625),  #U
            mp.Vector3(0.5  ,0.5  ,0.5  ),  #L
            mp.Vector3(0.0  ,0.0  ,0.0  ),  #Gamma
            mp.Vector3(0.0  ,0.5  ,0.5  ),  #X
            mp.Vector3(0.25 ,0.75 ,0.5  ),  #W
            mp.Vector3(0.375,0.75 ,0.375),  #K
            mp.Vector3()]  

res0 = mp.Vector3(12,20,28)
n0 = 1.5
tol = 0.001
mesh_size = 3
L = math.sqrt(3)/4
n = math.sqrt(32)
r = 0.0725
m = mp.Medium(epsilon=n*n)

geometry = [mp.Cylinder(center=mp.Vector3(0,0,0),radius=r, material=m, height=L, axis=mp.Vector3(1,1,1)),
           mp.Cylinder(center=mp.Vector3(0.5,0,0),radius=r, material=m, height=L, axis=mp.Vector3(3,-1,-1)),
           mp.Cylinder(center=mp.Vector3(0,0.5,0),radius=r, material=m, height=L, axis=mp.Vector3(-1,3,-1)),
           mp.Cylinder(center=mp.Vector3(0,0,0.5),radius=r, material=m, height=L, axis=mp.Vector3(-1,-1,3))]

a_x = math.sqrt(0.5)
geometry_lattice = mp.Lattice(
    basis_size=mp.Vector3(a_x,a_x,a_x),
    basis1=mp.Vector3(0,1,1),
    basis2=mp.Vector3(1,0,1),
    basis3=mp.Vector3(1,1,0))

#sqrt_half = math.sqrt(0.5)
#geometry_lattice = mp.Lattice(
#    basis_size=mp.Vector3(sqrt_half, sqrt_half, sqrt_half),
#    basis1=mp.Vector3(0, 1, 1),
#    basis2=mp.Vector3(1, 0, 1),
#    basis3=mp.Vector3(1, 1)
#)
k_points = mp.interpolate(1, k_points)

ms = mpb.ModeSolver(num_bands=num_bands,
                    k_points=k_points,
                    geometry=geometry,
                    geometry_lattice=geometry_lattice,
                    resolution=res0,
                    mesh_size = mesh_size)
#mpb.ModeSolver.optimize_grid_size()
ms.run()
ms.output_epsilon()

In [ ]:
ms.band_range_data

In [ ]:
!rm PI.dat
!python PIT.py >& PI.out
!grep freqs PI.out >> PI.dat

In [ ]:
!rm *.h5
!rm *.vtk
!python PIT.py 
!h5tovtk PIT-epsilon.h5

In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt
num_bands=50
nip_raw = []
k_raw = []
omega_raw = []

with open('PI.dat') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    next(csv_file)
    for row in csv_reader:
        nip_raw.append(row[1])
        k_raw.append(row[2:5])
        omega_raw.append(row[6:num_bands+6])
        

In [ ]:
N=np.size(nip_raw)
nip=np.zeros((N))
k=np.zeros((N,3))
omega=np.zeros((N,num_bands))
for i in range(0,N):
    nip[i] = float(nip_raw[i])
    for j in range(0,3):
        k[i][j] = float(k_raw[i][j])
        
    for j in range(0,num_bands):
        omega[i][j] = float(omega_raw[i][j])

plt.plot(nip,omega[:,:])

In [1]:
#%%writefile PIT.py
import math
import meep as mp
from meep import mpb
num_bands = 15
ax = math.sqrt(2)/2.0
ay = math.sqrt(6)/2.0
az = math.sqrt(3)
L = math.sqrt(3)/4.0
hs = math.sqrt(3)/12.0
na = 1
r = 0.0725
geometry_lattice = mp.Lattice(size=mp.Vector3(na*ax,na*ay,na*az))
#v1 = mp.Vector3(1)

v2 = mp.Vector3(0.5*ax,0.5*ay,0)
v1 = mp.Vector3(0,ay*1.0/3.0,az*1.0/3.0)
v5 = mp.Vector3(ax*0.5,ay*5.0/6.0,az*1.0/3.0)
v3 = mp.Vector3(ax*0.5,ay*1.0/6.0,az*2.0/3.0)
v4 = mp.Vector3(0,ay*2.0/3.0,az*2.0/3.0)
res0 = 20
n0 = 1.5
mesh_size = 3
n = math.sqrt(32)
r = 0.0725
m = mp.Medium(epsilon=n*n)
c1 = mp.Vector3(0,-math.sqrt(6)/12,-hs/2)
c2 = mp.Vector3(math.sqrt(2)/8,math.sqrt(6)/24,-hs/2)
c3 = mp.Vector3(-math.sqrt(2)/8,math.sqrt(6)/24,-hs/2)
c4 = mp.Vector3(0,0,L/2)
geometry = [mp.Cylinder(center=c1,radius=r, material=m, height=L, axis=c1),
            mp.Cylinder(center=c2,radius=r, material=m, height=L, axis=c2),
            mp.Cylinder(center=c3,radius=r, material=m, height=L, axis=c3),
            mp.Cylinder(center=c4,radius=r, material=m, height=L, axis=c4),
            mp.Cylinder(center=c1+v1,radius=r, material=m, height=L, axis=c1),
            mp.Cylinder(center=c2+v1,radius=r, material=m, height=L, axis=c2),
            mp.Cylinder(center=c3+v1,radius=r, material=m, height=L, axis=c3),
            mp.Cylinder(center=c4+v1,radius=r, material=m, height=L, axis=c4),
            mp.Cylinder(center=c1+v2,radius=r, material=m, height=L, axis=c1),
            mp.Cylinder(center=c2+v2,radius=r, material=m, height=L, axis=c2),
            mp.Cylinder(center=c3+v2,radius=r, material=m, height=L, axis=c3),
            mp.Cylinder(center=c4+v2,radius=r, material=m, height=L, axis=c4),
            mp.Cylinder(center=c1+v3,radius=r, material=m, height=L, axis=c1),
            mp.Cylinder(center=c2+v3,radius=r, material=m, height=L, axis=c2),
            mp.Cylinder(center=c3+v3,radius=r, material=m, height=L, axis=c3),
            mp.Cylinder(center=c4+v3,radius=r, material=m, height=L, axis=c4),
            mp.Cylinder(center=c1+v4,radius=r, material=m, height=L, axis=c1),
            mp.Cylinder(center=c2+v4,radius=r, material=m, height=L, axis=c2),
            mp.Cylinder(center=c3+v4,radius=r, material=m, height=L, axis=c3),
            mp.Cylinder(center=c4+v4,radius=r, material=m, height=L, axis=c4),
            mp.Cylinder(center=c1+v5,radius=r, material=m, height=L, axis=c1),
            mp.Cylinder(center=c2+v5,radius=r, material=m, height=L, axis=c2),
            mp.Cylinder(center=c3+v5,radius=r, material=m, height=L, axis=c3),
            mp.Cylinder(center=c4+v5,radius=r, material=m, height=L, axis=c4)]

#geometry2 = mp.geometric_objects_duplicates(v1,1,1,geometry_unit)
#geometry3 = mp.geometric_objects_duplicates(mp.Vector3(0,math.sqrt(6)/2/ay,0),1,1,geometry_unit)
#geometry4 = mp.geometric_objects_duplicates(v3,1,1,geometry_unit)
#geometry5 = mp.geometric_objects_duplicates(v3,1,1,geometry2)
#geometry6 = mp.geometric_objects_duplicates(v2,1,1,geometry4)
geometry = mp.geometric_objects_lattice_duplicates(geometry_lattice, geometry, ax,ay,az)
geometry.append(mp.Cylinder(center=c1,radius=r, material=mp.air, height=L, axis=c1))
k_points = [mp.Vector3(),
            mp.Vector3(0,0.5),
            mp.Vector3(0.5,0.5),
            mp.Vector3(),
            mp.Vector3(0,0.5),
            mp.Vector3(0.5,0.5,0.5),
            mp.Vector3()]          # Gamma
k_points = mp.interpolate(4, k_points)
k_points = [mp.Vector3()]
ms = mpb.ModeSolver(num_bands=num_bands,
                    k_points=k_points,
                    geometry=geometry,
                    geometry_lattice=geometry_lattice,
                    resolution=res0,
                    mesh_size = mesh_size)


In [2]:

#mpb.ModeSolver.optimize_grid_size()
ms.run()
ms.output_epsilon()
!h5tovtk epsilon.h5 

Initializing eigensolver data
Computing 15 bands with 1e-07 tolerance
1 k-points
  Vector3<0.0, 0.0, 0.0>
elapsed time for initialization: 0.0908770561218
elapsed time for k point: 6.1603000164
total elapsed time for run: 6.25145792961
done
Outputting epsilon.h5...


In [3]:
!h5tovtk epsilon.h5 

In [ ]:
!python PIT.py >& PI.out
!rm PI.dat
#!rm *.h5
!grep freqs PI.out >> PI.dat

import csv
import numpy as np
import matplotlib.pyplot as plt
num_bands=15
nip_raw = []
k_raw = []
omega_raw = []

with open('PI.dat') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    next(csv_file)
    for row in csv_reader:
        nip_raw.append(row[1])
        k_raw.append(row[2:5])
        omega_raw.append(row[6:num_bands+6])
        
N=np.size(nip_raw)
nip=np.zeros((N))
k=np.zeros((N,3))
omega=np.zeros((N,num_bands))
for i in range(0,N):
    nip[i] = float(nip_raw[i])
    for j in range(0,3):
        k[i][j] = float(k_raw[i][j])
        
    for j in range(0,num_bands):
        omega[i][j] = float(omega_raw[i][j])

plt.plot(nip,omega[:,:])

In [ ]:
plt.plot(nip,omega[:,1:13])

In [ ]:
plt.plot(nip,omega[:,:])

In [ ]:
!python PIT.py 
!h5tovtk PIT-epsilon.h5

In [ ]:
mp.Vector3(1)+mp.Vector3(0,1)